In [1]:
from pathlib import Path

main_folder = Path().cwd()
main_folder

PosixPath('/home/max/DEV/TEST/python-geoserver')

In [4]:
from fsspec import get_fs_token_paths
from owslib.wms import WebMapService


def get_wms_data(
        wms_url: str,
        service_version: str,
        layer_name: str,
        srs: str = 'EPSG:4326',
        width: int = 300,
        high: int = 300,
) -> dict:
    """Get thumbnail from WMS layer, it's saved into selected folder and also return
    the saved path.
    Args:
        wms_url: String.
        service_version: String.
        layer_name: String.
        srs: String.
        width: Integer.
        high: Integer.
    Returns:
        dict
    """
    # Read WMS
    wms = WebMapService(url=wms_url, version=service_version)

    # Get bounding box from selected layer
    layer = wms[layer_name]
    bbox = layer.boundingBoxWGS84

    # Create thumbnail
    img = wms.getmap(
        layers=[layer_name],
        srs=srs,
        bbox=bbox,
        size=(width, high),
        format='image/jpeg',
        transparent=True
    )

    # Make destination folder
    destination_folder = main_folder.joinpath('media')
    fs, fs_token, paths = get_fs_token_paths(destination_folder)
    fs.mkdirs(path=destination_folder, exist_ok=True)

    # Put thumbnail into destinantion folder
    img_path = Path(f'{destination_folder}/{layer.title}.jpg')
    out = open(img_path, 'wb')
    out.write(img.read())
    out.close()

    data = {
        "wms-title": layer.title,
        "wms-bbox": bbox,
        "wms-img": img_path
    }

    return data


In [5]:
layer = get_wms_data(
    wms_url="https://geoserver.massimilianomoraca.me/geoserver/MassimilianoMoraca/wms",
    service_version="1.3.0",
    layer_name="Beirut_20200724"
)
layer

{'wms-title': 'Beirut_20200724',
 'wms-bbox': (35.49074546110199,
  33.88620432137868,
  35.54600838012123,
  33.92160376571072),
 'wms-img': PosixPath('/home/max/DEV/TEST/python-geoserver/media/Beirut_20200724.jpg')}